In [2]:
import spacy

## Prodigy

In our patents, a lot of named entities are abbreviated.
For example, `WI-FI Direct` is mentioned as `WFD`, or `P2P Group Owner` as `GO`.

Our original model does not recognize these abbreviations and therefore a huge part of the entities are ignored.
So we will fine-tune our model using prodigy

In [6]:
nlp = spacy.load("spacy_output/model-best")

doc = nlp("Wi-Fi Direct (registered trademark, which will be hereinafter referred to as WFD) corresponding to a technology for directly performing a communication based on a wireless LAN between communication devices without intermediation of an access point (hereinafter referred to as AP) is standardized in Wi-Fi Alliance serving as a wireless LAN industry group.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

/Users/gaetanserre/miniconda3/envs/NLP/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


We create a train dataset for fine-tuning our ner model.

In [5]:
!prodigy ner.correct fine_tune_g06k2 spacy_output/model-best G06K.txt --loader txt --label TECH

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Using 1 label(s): TECH
Added dataset fine_tune_g06k2 to database SQLite.
/Users/gaetanserre/miniconda3/envs/NLP/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

^C


We export our dataset.

In [6]:
!prodigy db-out fine_tune_g06k terms_g06k

✔ Exported 35 annotations from 'fine_tune_g06k' in database SQLite
/Users/gaetanserre/Documents/Projects/patent_ner_linking/terms_g06k/fine_tune_g06k.jsonl


Now let's fine-tune our model!

In [10]:
!prodigy train ./prodigy_output/ --ner fine_tune_g06k --base-model spacy_output/model-best --gpu-id 0

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

========================= Generating Prodigy config =========================
Traceback (most recent call last):
  File "/home/gaetan/miniconda3/envs/ML/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/gaetan/miniconda3/envs/ML/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/gaetan/miniconda3/envs/ML/lib/python3.9/site-packages/prodigy/__main__.py", line 61, in <module>
    controller = recipe(*args, use_plac=True)
  File "cython_src/prodigy/core.pyx", line 329, in prodigy.core.recipe.recipe_decorator.recipe_proxy
  File "/home/gaetan/miniconda3/envs/ML/lib/python3.9/site-packages/plac_core.py", line 367, in call
    cmd, result = parser.consume(arglist)
  File "/home/gaetan/miniconda3/envs/ML/lib/python3.9/site-packages/plac_core.py", line 232, in consume
    return cmd, self.func(*(args + varargs + extraopts), **k

Now our refined model should recognize better the abbreviations.

In [3]:
nlp = spacy.load("prodigy_output/model-best")

doc = nlp("According to the WFD, the communication is performed when one of the \
           communication devices that directly perform the wireless LAN communication \
           operates as the AP. According to the WFD, a role of the device that operates \
           as the AP will be referred to as P2P Group Owner (hereinafter, referred to as GO). \
           On the other hand, a role of the device that participates in a network generated by \
           the GO will be referred to as P2P Client (hereinafter, referred to as CL). \
           According to the WFD, a communication parameter necessary for participating in \
           the network generated by the GO is shared between the devices by transmitting the \
           communication parameter from the GO to the CL, and thereafter, the wireless \
           communication according to the WFD is executed on the basis of the shared communication parameter.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

/Users/gaetanserre/miniconda3/envs/NLP/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
